### Install the required packages

In [ ]:
%pip install python-dotenv
%pip install requests
%pip install pip-system-certs


### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import sys
sys.path.append('../common')
from helper import Helper

load_dotenv()

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "text-chunking/microsoft/simple", api_key, region_override)

### Test Case 364782: ST_TC_WB_1204: Verify Chunking with Invalid Algorithm

In [ ]:
helper.log("Testcase 364782: ST_TC_WB_1204: Verify Chunking with Invalid Algorithm | Validating only status code")
response = helper.post(
    "/simple",
    {
        "extractedFileContent":"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
        "format":"text",
        "algorithm":"Testinvalid_Algo",
        "tokensPerChunk":50,
        "overlapTokens":5
    }
)
helper.log ("Response  is ", response.text)
assert response.status_code == 400
#assert response.json["Details"] == "Body of the request does not conform to the definition which is associated with the content type application/json. Path:algorithm Message: Value \"Testinvalid_Algo\" is not defined in enum. Line: 1, Position: 518 SchemaId: #/components/schemas/simpleTextChunkerRequest/properties/algorithm"

### Test Case 364783: ST_TC_WB_1204: Verify Chunking with Invalid tokensPerChunk

In [ ]:
helper.log("Test Case 364783: ST_TC_WB_1204: Verify Chunking with Invalid tokensPerChunk | Validating only status code and Message")
response = helper.post(
    "/simple",
    {
        "extractedFileContent":"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
        "format":"text",
        "algorithm":"line",
        "tokensPerChunk": -10,
        "overlapTokens":10
    }
)

assert response.status_code == 400
assert response.json["message"] == "Tokens per Chunk should be an integer greater than 0."

### Test Case 364784: ST_TC_WB_1204: Verify Chunking with Invalid overlapTokens

In [ ]:
helper.log("Test Case 364784: ST_TC_WB_1204: Verify Chunking with Invalid overlapTokens| Validating only status code and Message")
response = helper.post(
    "/simple",
    {
        "extractedFileContent":"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
        "format":"text",
        "algorithm":"line",
        "tokensPerChunk": 10,
        "overlapTokens":-10
    }
)
helper.log ("Response - ", response.text)
assert response.status_code == 400
assert response.json["message"] == "Overlap Tokens should be an integer greater than 0 and less than Token per Chunk."

### Test Case 364781: ST_TC_WB_1204: Verify Chunking with Invalid Format

In [ ]:
helper.log ("Test Case 364781: ST_TC_WB_1204: Verify Chunking with Invalid Format | Validating Status code only")
response = helper.post(
    "/simple",
    {
        "extractedFileContent":"Lorem ipsum dolor sit amet, consectetur adipiscing elit, sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.",
        "format":"Invalid_format",
        "algorithm":"line",
        "tokensPerChunk": 10,
        "overlapTokens": 2
    }
)
helper.log ("Response - ", response.text)
assert response.status_code == 400
#assert response.json["message"] == "Body of the request does not conform to the definition which is associated with the content type application/json. Path:format Message: Value \"Invalid_format\" is not defined in enum. Line: 1, Position: 497 SchemaId: #/components/schemas/simpleTextChunkerRequest/properties/format"

### Test Case 364785: ST_TC_WB_1204: Verify Chunking with Empty extractedFileContent

In [ ]:
helper.log("Test Case 364785: ST_TC_WB_1204: Verify Chunking with Empty extractedFileContent | Validating only Status code")
response = helper.post(
    "/simple",
    {
        "extractedFileContent":"",
        "format":"text",
        "algorithm":"line",
        "tokensPerChunk": 10,
        "overlapTokens": 2
    }
)
helper.log("Response - ", response.text)
assert response.status_code == 400
#assert response.json["message"] == "Body of the request does not conform to the definition which is associated with the content type application/json. Path:extractedFileContent Message: String '' is less than minimum length of 1. Line: 1, Position: 26 SchemaId: #/components/schemas/simpleTextChunkerRequest/properties/extractedFileContent"